In [3]:
import cv2
import numpy as np
from ultralytics import YOLO
from collections import defaultdict

# ==================== 配置参数 ====================
VIDEO_PATH = "highway_n.avi"              # 输入视频路径
OUTPUT_PATH = "output_video_d_di.mp4"        # 输出视频路径
CLASS_NAMES = [2, 5, 7]                 # 车辆类别ID（COCO: car=2, bus=5, truck=7）
TRACKER_CONFIG = "botsort.yaml"         # 跟踪算法配置文件

# 车道配置（每个车道定义一条计数线和方向检测范围）
LANE_CONFIG = [
    {
        "line": [(0, 850), (1920, 850)],    # 车道1计数线（起点和终点坐标）
        "direction_roi": (0, 700, 1920, 1010)  # 方向检测ROI区域 (x1, y1, x2, y2)
    },
    {
        "line": [(0, 600), (1920, 600)],    # 车道2计数线
        "direction_roi": (0, 300, 1920, 700)
    }
]

# ==================== 核心逻辑 ====================
def main():
    # 加载模型
    model = YOLO("yolov8l.pt")
    
    # 打开视频
    cap = cv2.VideoCapture(VIDEO_PATH)
    assert cap.isOpened(), "视频打开失败"
    
    # 获取视频信息
    fps = cap.get(cv2.CAP_PROP_FPS)
    width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    
    # 初始化视频写入
    writer = cv2.VideoWriter(OUTPUT_PATH, cv2.VideoWriter_fourcc(*'mp4v'), fps, (width, height))
    
    # 初始化统计信息
    track_history = defaultdict(list)      # 轨迹记录 {track_id: [(x, y)]}
    counted_ids = defaultdict(set)          # 已计数车辆ID {lane_id: set()}
    lane_counts = defaultdict(lambda: {"up": 0, "down": 0})  # 车道计数 {lane_id: {"up": N, "down": M}}

    while cap.isOpened():
        success, frame = cap.read()
        if not success:
            break

        # 使用跟踪模式推理
        results = model.track(
            frame,
            persist=True,
            tracker=TRACKER_CONFIG,
            device="coda",              # 显式指定GPU设备
            classes=CLASS_NAMES,
            verbose=False
        )

        # 获取检测结果
        if results[0].boxes.id is not None:
            boxes = results[0].boxes.xyxy.cpu().numpy()
            track_ids = results[0].boxes.id.cpu().numpy().astype(int)
            class_ids = results[0].boxes.cls.cpu().numpy().astype(int)

            for box, track_id, class_id in zip(boxes, track_ids, class_ids):
                x1, y1, x2, y2 = box
                center_x = int((x1 + x2) / 2)
                center_y = int((y1 + y2) / 2)

                # 记录轨迹（保留最近15个点）
                track = track_history[track_id]
                track.append((center_x, center_y))
                if len(track) > 15:
                    track.pop(0)

                # 判断车辆所在车道
                current_lane = None
                for lane_id, lane in enumerate(LANE_CONFIG):
                    roi_x1, roi_y1, roi_x2, roi_y2 = lane["direction_roi"]
                    if roi_x1 < center_x < roi_x2 and roi_y1 < center_y < roi_y2:
                        current_lane = lane_id
                        break

                if current_lane is not None:
                    # 获取当前车道的计数线
                    line_start, line_end = LANE_CONFIG[current_lane]["line"]
                    line_mask = np.zeros(frame.shape[:2], dtype=np.uint8)
                    cv2.line(line_mask, line_start, line_end, 255, 2)

                    # 检测是否与计数线相交
                    if len(track) >= 2:
                        prev_x, prev_y = track[-2]
                        curr_x, curr_y = center_x, center_y

                        # 生成移动线段
                        track_line_mask = np.zeros(frame.shape[:2], dtype=np.uint8)
                        cv2.line(track_line_mask, (prev_x, prev_y), (curr_x, curr_y), 255, 2)

                        # 判断线段是否相交
                        intersection = cv2.bitwise_and(line_mask, track_line_mask)
                        if np.any(intersection > 0):
                            # 判断方向（根据y坐标变化）
                            direction = "up" if curr_y < prev_y else "down"

                            if track_id not in counted_ids[current_lane]:
                                lane_counts[current_lane][direction] += 1
                                counted_ids[current_lane].add(track_id)

                    # 绘制检测框和轨迹
                    color = (0, 255, 0) if current_lane == 0 else (255, 0, 0)
                    cv2.rectangle(frame, (int(x1), int(y1)), (int(x2), int(y2)), color, 2)
                    cv2.putText(frame, f"ID: {track_id}", (int(x1), int(y1)-10), 
                               cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 2)
                    for point in track:
                        cv2.circle(frame, point, 3, color, -1)

        # 绘制车道线和统计信息
        for lane_id, lane in enumerate(LANE_CONFIG):
            # 绘制计数线
            cv2.line(frame, lane["line"][0], lane["line"][1], (0, 0, 255), 2)
            
            # 绘制方向ROI区域
            roi_x1, roi_y1, roi_x2, roi_y2 = lane["direction_roi"]
            cv2.rectangle(frame, (roi_x1, roi_y1), (roi_x2, roi_y2), (255, 255, 0), 1)
            
            # 显示车道计数
            count_up = lane_counts[lane_id]["up"]
            count_down = lane_counts[lane_id]["down"]
            cv2.putText(frame, f"Lane {lane_id+1}: Up={count_up} Down={count_down}", 
                       (20, 40 + lane_id*30), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 255), 2)

        # 写入视频帧
        writer.write(frame)
        
        # 实时显示（按ESC退出）
        cv2.imshow("Multi-Lane Counting", frame)
        if cv2.waitKey(1) == 27:
            break

    # 释放资源
    cap.release()
    writer.release()
    cv2.destroyAllWindows()
    print("统计完成！")
    for lane_id in lane_counts:
        print(f"车道 {lane_id+1}: 上行={lane_counts[lane_id]['up']} 下行={lane_counts[lane_id]['down']}")

if __name__ == "__main__":
    main()

统计完成！
车道 1: 上行=17 下行=2515
车道 2: 上行=175 下行=1456
